# 範例 13：使用向量搜尋的 RAG 系統

這個範例使用向量嵌入（Embedding）來搜尋最相關的文件！

## 什麼是向量嵌入？

向量嵌入是把文字轉換成一串數字（向量）的技術：

```
"我喜歡程式設計" → [0.2, -0.5, 0.8, 0.1, ...]
"我愛寫程式"    → [0.3, -0.4, 0.7, 0.2, ...]  ← 意思相近，向量也相近！
"今天天氣很好"  → [-0.1, 0.6, -0.2, 0.9, ...] ← 意思不同，向量差很多
```

## 為什麼要用向量搜尋？
- 關鍵字搜尋：「狗」搜不到「犬」（雖然意思一樣）
- 向量搜尋：「狗」可以找到「犬」（因為意思相近，向量相近）

## 學習目標
- 了解向量嵌入的概念
- 學會計算向量相似度
- 實作向量搜尋的 RAG 系統

## 前置需求
- LM Studio 運行中，已載入嵌入模型
- 安裝套件：`pip install numpy openai`

## Step 1: 匯入套件並設定

In [ ]:
import numpy as np
from openai import OpenAI

# 初始化客戶端
client = OpenAI(
    base_url="http://localhost:1234/v1",
    api_key="not-needed"
)

print("客戶端已就緒！")

## Step 2: 準備知識庫文件

In [ ]:
# 知識庫文件
DOCUMENTS = [
    "Python 是一種簡單易學的程式語言，適合初學者入門。",
    "JavaScript 是網頁開發的核心語言，可以讓網頁產生互動效果。",
    "機器學習讓電腦能從資料中學習，不需要明確的程式指令。",
    "深度學習是機器學習的一個分支，使用神經網路來處理複雜問題。",
    "自然語言處理（NLP）讓電腦能理解和生成人類語言。",
    "RAG 技術結合了資訊檢索和文字生成，提高 AI 回答的準確性。",
]

print("知識庫文件：")
for i, doc in enumerate(DOCUMENTS, 1):
    print(f"{i}. {doc}")

## Step 3: 理解向量嵌入

In [ ]:
def get_embedding(text):
    """
    取得文字的向量表示（Embedding）

    向量嵌入是什麼？
    - 把文字轉換成一串數字（向量）
    - 意思相近的文字，向量也會相近
    - 這樣電腦就能「理解」文字的意義
    """
    response = client.embeddings.create(
        model="text-embedding-nomic-embed-text-v1.5",  # 嵌入模型
        input=text
    )
    return response.data[0].embedding

In [ ]:
# 觀察向量長什麼樣子
try:
    sample_text = "Python 程式設計"
    sample_embedding = get_embedding(sample_text)
    
    print(f"文字：{sample_text}")
    print(f"向量維度：{len(sample_embedding)}")
    print(f"向量前 10 個數值：{sample_embedding[:10]}")
except Exception as e:
    print(f"錯誤：{e}")
    print("請確認 LM Studio 已載入嵌入模型（如 nomic-embed-text）")

## Step 4: 計算向量相似度

### 餘弦相似度（Cosine Similarity）

衡量兩個向量方向的相似程度：
- 1 = 完全相同方向
- 0 = 完全無關
- -1 = 完全相反

In [ ]:
def cosine_similarity(vec1, vec2):
    """
    計算兩個向量的餘弦相似度

    餘弦相似度是什麼？
    - 衡量兩個向量方向的相似程度
    - 值介於 -1 到 1 之間
    - 1 表示完全相同，0 表示無關，-1 表示完全相反
    """
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    
    # 餘弦相似度公式：(A·B) / (|A| × |B|)
    dot_product = np.dot(vec1, vec2)           # 內積
    norm1 = np.linalg.norm(vec1)               # 向量 1 的長度
    norm2 = np.linalg.norm(vec2)               # 向量 2 的長度
    
    return dot_product / (norm1 * norm2)

In [ ]:
# 測試相似度計算
try:
    text1 = "我喜歡寫程式"
    text2 = "程式設計很有趣"
    text3 = "今天天氣很好"
    
    emb1 = get_embedding(text1)
    emb2 = get_embedding(text2)
    emb3 = get_embedding(text3)
    
    print("相似度測試：")
    print(f"'{text1}' vs '{text2}'：{cosine_similarity(emb1, emb2):.4f}")
    print(f"'{text1}' vs '{text3}'：{cosine_similarity(emb1, emb3):.4f}")
    print("\n意思相近的句子，相似度比較高！")
except Exception as e:
    print(f"錯誤：{e}")

## Step 5: 初始化知識庫向量

In [ ]:
# 儲存文件的向量表示
document_embeddings = []

def initialize_knowledge_base():
    """
    初始化知識庫：為所有文件計算向量
    """
    global document_embeddings
    print("正在初始化知識庫...")

    for i, doc in enumerate(DOCUMENTS, 1):
        print(f"  處理文件 {i}/{len(DOCUMENTS)}...")
        embedding = get_embedding(doc)
        document_embeddings.append(embedding)

    print(f"完成！已載入 {len(DOCUMENTS)} 份文件")

In [ ]:
# 初始化（這可能需要一些時間）
try:
    initialize_knowledge_base()
except Exception as e:
    print(f"初始化失敗：{e}")
    print("請確認 LM Studio 已載入嵌入模型")

## Step 6: 實作向量搜尋

In [ ]:
def vector_search(query, top_k=2):
    """
    向量搜尋：找出與問題最相關的文件

    參數：
        query: 使用者的問題
        top_k: 要返回的文件數量

    回傳：
        最相關的文件列表（包含相似度分數）
    """
    # 取得問題的向量
    query_embedding = get_embedding(query)

    # 計算與每份文件的相似度
    similarities = []
    for i, doc_embedding in enumerate(document_embeddings):
        similarity = cosine_similarity(query_embedding, doc_embedding)
        similarities.append((similarity, DOCUMENTS[i], i))

    # 按相似度排序，取前 k 個
    similarities.sort(reverse=True)
    return similarities[:top_k]

In [ ]:
# 測試向量搜尋
test_query = "什麼是 AI 學習技術？"
print(f"查詢：{test_query}")
print("-" * 50)

try:
    results = vector_search(test_query, top_k=3)
    print("搜尋結果（按相似度排序）：")
    for similarity, doc, idx in results:
        print(f"\n相似度：{similarity:.4f}")
        print(f"文件：{doc}")
except Exception as e:
    print(f"搜尋失敗：{e}")

## Step 7: 完整的向量 RAG 系統

In [ ]:
def rag_with_vector_search(question):
    """
    使用向量搜尋的 RAG 聊天
    """
    print(f"[向量 RAG] 處理問題：{question}")
    
    # 搜尋相關文件
    search_results = vector_search(question, top_k=2)
    relevant_docs = [doc for _, doc, _ in search_results]
    
    print(f"[向量 RAG] 找到 {len(relevant_docs)} 筆相關文件")
    for similarity, doc, _ in search_results:
        print(f"  - 相似度 {similarity:.3f}: {doc[:30]}...")

    # 建立提示詞
    context = "\n".join([f"- {doc}" for doc in relevant_docs])

    prompt = f"""請根據以下參考資料回答問題：

參考資料：
{context}

問題：{question}

請用繁體中文簡潔回答："""

    # 呼叫 AI
    response = client.chat.completions.create(
        model="gpt-oss-120b",
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content, search_results

In [ ]:
# 測試向量 RAG
question = "NLP 是做什麼的？"

try:
    answer, sources = rag_with_vector_search(question)
    
    print("\n" + "=" * 50)
    print(f"問題：{question}")
    print(f"\n回答：{answer}")
    print("\n參考來源：")
    for similarity, doc, _ in sources:
        print(f"  [{similarity:.3f}] {doc}")
except Exception as e:
    print(f"錯誤：{e}")

## 向量搜尋 vs 關鍵字搜尋

| 特點 | 關鍵字搜尋 | 向量搜尋 |
|------|------------|----------|
| 速度 | 很快 | 較慢 |
| 語義理解 | 無 | 有 |
| 同義詞處理 | 需要手動設定 | 自動處理 |
| 適用場景 | 精確搜尋 | 模糊搜尋 |

### 範例比較：
- 問題：「AI 學習技術」
- 關鍵字搜尋：找不到（沒有完全匹配的關鍵字）
- 向量搜尋：找到「機器學習」「深度學習」（意思相近）

## 練習區

In [ ]:
# 試著問不同的問題，觀察向量搜尋的效果
my_question = "如何讓電腦理解人話？"  # 修改這個問題

try:
    answer, sources = rag_with_vector_search(my_question)
    print(f"\n問題：{my_question}")
    print(f"回答：{answer}")
except Exception as e:
    print(f"錯誤：{e}")

## 重點回顧

1. **向量嵌入**：把文字轉成數字向量，意思相近的文字向量也相近
2. **餘弦相似度**：計算兩個向量的相似程度（-1 到 1）
3. **向量搜尋流程**：
   - 預先計算所有文件的向量
   - 查詢時計算問題的向量
   - 找出最相似的文件
4. **優點**：能找到意思相近但字詞不同的文件

## 下一步

在下一個範例中，我們將學習如何建立文件問答系統！